In [1]:
import random
from typing import Optional, Union, Dict, List,Any
import json
from loguru import logger

In [2]:
from vj.agents.core import BaseAgent,AgentError
from vj.tools.tool import Tool
from vj.agents.templates import ReactAgentTemplate
from vj.tools.tool import CalculatorTool

In [3]:
class ReactAgent(BaseAgent):
    """
    React Agent implementation following thought-action-observation cycle
    """

    def __init__(
        self,
        tools: List[Tool],
        model: callable,
        max_steps: int = 10,
        template: Optional[ReactAgentTemplate] = None,
        temperature: float = 0.7,
    ):
        super().__init__(
            tools=tools,
            model=model,
            max_steps=max_steps,
            name="react_agent",
            description="A React-style agent that follows thought-observation-action cycle",
        )
        self.template = template or ReactAgentTemplate()
        self.temperature = temperature

    def generate_action(self, task: str) -> dict:
        """
        Generate next action based on thought process

        Args:
            task (str): The current task

        Returns:
            dict: Action dictionary with tool and input
        """
        # Format history and create prompt
        history = self._format_history()

        # Create system message
        system_prompt = (
            self.template.DEFAULT_TEMPLATE.split("[prompts]")[1]
            .split("system = '''")[1]
            .split("'''")[0]
        )

        # Build prompt with task and history
        prompt = f"""
        Task: {task}
        Previous Steps: {history}
        Current Step: {self.current_step + 1}

        Think through this step by step. Available tools:
        {self._format_tools()}

        Respond with:
        Thought: [your reasoning]
        Action: [tool_name] [parameters]
        """

        # Generate response
        response = self.model(system_prompt, prompt, temperature=self.temperature)

        # Parse response to extract action
        try:
            # Extract action from response
            thought, action = "", ""

            if "Thought:" in response:
                thought_parts = (
                    response.split("Thought:")[1].split("Action:")[0].strip()
                )
                thought = thought_parts

            if "Action:" in response:
                action_parts = response.split("Action:")[1].strip().split()
                if len(action_parts) >= 2:
                    tool_name = action_parts[0]
                    tool_input = " ".join(action_parts[1:])

                    return {"thought": thought, "tool": tool_name, "input": tool_input}

            raise AgentError("Failed to parse action from response")

        except Exception as e:
            logger.error(f"Failed to generate action: {str(e)}")
            raise AgentError(f"Action generation failed: {str(e)}")

    def execute_action(self, action: dict) -> Any:
        """
        Execute the generated action using available tools

        Args:
            action (dict): Action dictionary with tool and input

        Returns:
            Any: Result of tool execution
        """
        try:
            tool_name = action["tool"]
            tool_input = action["input"]

            if tool_name == "final_answer":
                return {"observation": tool_input, "status": "final_answer"}

            if tool_name not in self.tools:
                return {
                    "observation": f"Error: Unknown tool '{tool_name}'",
                    "status": "error",
                }

            tool = self.tools[tool_name]
            result = tool.execute(tool_input)

            return {"observation": str(result), "status": "success"}

        except Exception as e:
            logger.error(f"Action execution failed: {str(e)}")
            return {"observation": f"Error: {str(e)}", "status": "error"}

    def is_final_answer(self, result: Any) -> bool:
        """
        Check if the result is a final answer

        Args:
            result (Any): The action execution result

        Returns:
            bool: True if this is the final answer
        """
        if isinstance(result, dict) and "status" in result:
            return result["status"] == "final_answer"
        return False

    def _format_history(self) -> str:
        """Format the agent's memory into a string"""
        if not self.memory.steps:
            return "No previous actions"

        history = []
        for step in self.memory.steps:
            action = step["action"]
            result = step["result"]

            thought = action.get("thought", "")
            tool = action.get("tool", "")
            tool_input = action.get("input", "")

            observation = (
                result.get("observation", "")
                if isinstance(result, dict)
                else str(result)
            )

            history.append(f"Step {step['step'] + 1}:")
            history.append(f"Thought: {thought}")
            history.append(f"Action: {tool} {tool_input}")
            history.append(f"Observation: {observation}")

        return "\n".join(history)

    def _format_tools(self) -> str:
        """Format available tools for the prompt"""
        tool_descriptions = []
        for name, tool in self.tools.items():
            tool_descriptions.append(f"- {name}: {tool.description}")

        tool_descriptions.append(
            "- final_answer: Use this when you have the final answer to the task"
        )
        return "\n".join(tool_descriptions)

In [4]:
calculator = CalculatorTool()
tools = [calculator]

In [5]:
def model_fn(system_message, prompt, temperature=0.7):
    # This would normally call your LLM
    # For testing, return a dummy response
    if "plan" in prompt.lower():
        return """
        1. calculator 2+2
        2. calculator (4)*3
        """
    else:
        return """
        Thought: I need to calculate 2+2 first
        Action: calculator 2+2
        """

In [6]:
react_agent = ReactAgent(tools=tools, model=model_fn, max_steps=5)

In [7]:
react_result = react_agent.run("Calculate 2+2 and then multiply by 3")
print(f"React Agent Result: {react_result}")

React Agent Result: None


In [8]:
class PlannerAgent(BaseAgent):
    """
    A planner-executor agent that first creates a plan, then executes it step by step
    """

    def __init__(
        self,
        tools: List[Tool],
        model: callable,
        max_steps: int = 15,
        temperature: float = 0.7,
    ):
        super().__init__(
            tools=tools,
            model=model,
            max_steps=max_steps,
            name="planner_agent",
            description="An agent that creates a plan and then executes it",
        )
        self.temperature = temperature
        self.plan = []
        self.current_plan_step = 0

    def generate_action(self, task: str) -> dict:
        """
        Generate next action based on the plan or create a plan

        Args:
            task (str): The task to complete

        Returns:
            dict: Action dictionary
        """
        # If no plan exists, create one
        if not self.plan:
            self.plan = self._create_plan(task)
            logger.info(f"Created plan with {len(self.plan)} steps")

        # If we've completed the plan, return final answer
        if self.current_plan_step >= len(self.plan):
            return {"tool": "final_answer", "input": self._generate_final_answer(task)}

        # Get the current plan step
        current_step = self.plan[self.current_plan_step]
        self.current_plan_step += 1

        # Parse the tool and input
        try:
            parts = current_step.split(maxsplit=1)
            if len(parts) < 2:
                raise AgentError(f"Invalid plan step: {current_step}")

            tool_name = parts[0]
            tool_input = parts[1]

            return {
                "thought": f"Executing plan step {self.current_plan_step}: {current_step}",
                "tool": tool_name,
                "input": tool_input,
            }
        except Exception as e:
            logger.error(f"Failed to parse plan step: {str(e)}")
            return {
                "tool": "error",
                "input": f"Error in plan step {self.current_plan_step}: {str(e)}",
            }

    def execute_action(self, action: dict) -> Any:
        """
        Execute the action from the plan

        Args:
            action (dict): Action to execute

        Returns:
            Any: Result of the action
        """
        try:
            tool_name = action["tool"]
            tool_input = action["input"]

            if tool_name == "final_answer":
                return {"observation": tool_input, "status": "final_answer"}

            if tool_name == "error":
                return {"observation": tool_input, "status": "error"}

            if tool_name not in self.tools:
                return {
                    "observation": f"Error: Unknown tool '{tool_name}'",
                    "status": "error",
                }

            tool = self.tools[tool_name]
            result = tool.execute(tool_input)

            return {"observation": str(result), "status": "success"}

        except Exception as e:
            logger.error(f"Action execution failed: {str(e)}")
            return {"observation": f"Error: {str(e)}", "status": "error"}

    def is_final_answer(self, result: Any) -> bool:
        """Check if the result is a final answer"""
        if isinstance(result, dict) and "status" in result:
            return result["status"] == "final_answer"
        return False

    def _create_plan(self, task: str) -> List[str]:
        """
        Create a plan for completing the task

        Args:
            task (str): The task to plan for

        Returns:
            List[str]: List of plan steps
        """
        system_prompt = """You are a planning AI that creates step-by-step plans.
Your task is to break down complex problems into a series of tool-using steps.
Each step should use one tool and specify the input to that tool.
Format each step as: tool_name input_parameters"""

        prompt = f"""
Task: {task}

Available tools:
{self._format_tools()}

Create a detailed step-by-step plan to solve this task.
The plan should be a numbered list where each step uses a tool.
Format each step as: tool_name input_parameters
"""

        # Generate the plan
        response = self.model(system_prompt, prompt, temperature=self.temperature)

        # Parse the plan into steps
        plan_steps = []
        lines = response.strip().split("\n")

        for line in lines:
            # Skip empty lines and lines without tool calls
            line = line.strip()
            if not line:
                continue

            # Remove step numbers and bullets
            while line and not line[0].isalpha():
                line = line[1:].strip()

            # Remove step prefixes like "Step 1: "
            if ":" in line and line.split(":")[0].strip().lower().startswith("step"):
                line = line.split(":", 1)[1].strip()

            if line and " " in line:  # Ensure there's a tool and parameters
                plan_steps.append(line)

        return plan_steps

    def _generate_final_answer(self, task: str) -> str:
        """
        Generate a final answer based on the plan execution

        Args:
            task (str): The original task

        Returns:
            str: The final answer
        """
        system_prompt = (
            "You are an AI that summarizes results of a multi-step execution plan."
        )

        history = []
        for step in self.memory.steps:
            action = step["action"]
            result = step["result"]

            tool = action.get("tool", "")
            tool_input = action.get("input", "")

            observation = (
                result.get("observation", "")
                if isinstance(result, dict)
                else str(result)
            )

            history.append(f"Step {step['step'] + 1}:")
            history.append(f"Action: {tool} {tool_input}")
            history.append(f"Result: {observation}")

        prompt = f"""
Original task: {task}

Execution history:
{chr(10).join(history)}

Provide a concise final answer to the original task.
"""

        # Generate the final answer
        response = self.model(system_prompt, prompt, temperature=self.temperature)

        return response.strip()

    def _format_tools(self) -> str:
        """Format available tools for the prompt"""
        tool_descriptions = []
        for name, tool in self.tools.items():
            tool_descriptions.append(f"- {name}: {tool.description}")

        return "\n".join(tool_descriptions)

In [9]:
planner_agent = PlannerAgent(tools=tools, model=model_fn, max_steps=10)

In [10]:
planner_result = planner_agent.run("Calculate 2+2 and then multiply by 3")
print(f"Planner Agent Result: {planner_result}")

2025-03-31 23:34:35.495 | INFO     | __main__:generate_action:37 - Created plan with 2 steps


Planner Agent Result: {'observation': 'Thought: I need to calculate 2+2 first\n        Action: calculator 2+2', 'status': 'final_answer'}


In [11]:

class DummyLLM:
    """
    A dummy LLM implementation that returns random responses
    """

    def __init__(self, temperature: float = 0.7):
        self.temperature = temperature
        self._responses = [
            "I think we should use the calculator tool for this task.",
            "Let me search through the available tools.",
            "Based on my analysis, we should proceed step by step.",
            "I recommend using the following approach...",
            "The solution requires mathematical computation.",
            "Let's break this problem down into smaller parts.",
        ]

    def generate(
        self,
        prompt: str,
        system_message: Optional[str] = None,
        temperature: Optional[float] = None,
    ) -> Dict[str, Union[str, float]]:
        """
        Generate a random response with metadata

        Args:
            prompt (str): The input prompt
            system_message (Optional[str]): System message to guide generation
            temperature (Optional[float]): Override default temperature

        Returns:
            Dict containing response text and metadata
        """
        # Use random confidence score between 0.3 and 0.9
        confidence = random.uniform(0.3, 0.9)

        return {
            "text": random.choice(self._responses),
            "confidence": confidence,
            "tokens_used": random.randint(10, 50),
            "finish_reason": "stop",
        }

    def batch_generate(
        self,
        prompts: List[str],
        system_message: Optional[str] = None,
        temperature: Optional[float] = None,
    ) -> List[Dict[str, Union[str, float]]]:
        """
        Generate multiple random responses

        Args:
            prompts (List[str]): List of input prompts
            system_message (Optional[str]): System message to guide generation
            temperature (Optional[float]): Override default temperature

        Returns:
            List of response dictionaries
        """
        return [
            self.generate(prompt, system_message, temperature) for prompt in prompts
        ]